In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM,Dropout
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

/home/pc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Veri dosayamızı Pandas ile yüklüyoruz
df = pd.read_csv('data_ocak_train.txt')
dft =pd.read_csv('data_ocak_test.txt')

In [3]:
dft.head()
print(df.values)

[[-10]
 [-10]
 [-11]
 ...
 [  0]
 [  0]
 [ -1]]


In [4]:
# sadece en yüksek sıcaklık değerlerini alıyoruz
raw_datatr = df.values
raw_datats = dft.values

In [5]:
'''print(type(raw_date[0]))
from datetime import datetime
datetime_object = datetime.strptime(raw_date[0], '%d/%m/%Y')
data=[]
for i in range(len(raw_date)):
    temp=datetime.strptime(raw_date[i], '%d/%m/%Y').month
    tempy=datetime.strptime(raw_date[i], '%d/%m/%Y').year
    if temp==1 and tempy%3==0:
        print(datetime.strptime(raw_date[i], '%d/%m/%Y'))
        print(tempy)
        print("test data writing: ",raw_data[i])
        with open("data_ocak_test.txt", "a") as myfile:
            myfile.write(str(raw_data[i])+"\n")
    elif temp==1:
        print("Train data writing: ",raw_data[i])
        with open("data_ocak_train.txt", "a") as myfile:
            myfile.write(str(raw_data[i])+"\n")
        
'''

'print(type(raw_date[0]))\nfrom datetime import datetime\ndatetime_object = datetime.strptime(raw_date[0], \'%d/%m/%Y\')\ndata=[]\nfor i in range(len(raw_date)):\n    temp=datetime.strptime(raw_date[i], \'%d/%m/%Y\').month\n    tempy=datetime.strptime(raw_date[i], \'%d/%m/%Y\').year\n    if temp==1 and tempy%3==0:\n        print(datetime.strptime(raw_date[i], \'%d/%m/%Y\'))\n        print(tempy)\n        print("test data writing: ",raw_data[i])\n        with open("data_ocak_test.txt", "a") as myfile:\n            myfile.write(str(raw_data[i])+"\n")\n    elif temp==1:\n        print("Train data writing: ",raw_data[i])\n        with open("data_ocak_train.txt", "a") as myfile:\n            myfile.write(str(raw_data[i])+"\n")\n        \n'

In [6]:
# Pencere ebatımı belirleyip kayan pencere metoduyla veri stimizi oluşturuyoruz
timestep = 90
Xtr = []
Ytr = []
Xts = []
Yts = []
for i in range(len(raw_datatr)-(timestep)):
    Xtr.append(raw_datatr[i:i+timestep])
    Ytr.append(raw_datatr[i+timestep])
    
for i in range(len(raw_datats)-(timestep)):
    Xts.append(raw_datats[i:i+timestep])
    Yts.append(raw_datats[i+timestep])

In [7]:
print(len(Xts))

4184


In [8]:
# LSTM girişi için Veri şeklinde dönüşüm yapıyoruz
Xtr = np.asanyarray(Xtr)
Xtr = Xtr.reshape((Xtr.shape[0],Xtr.shape[1],1))


Xts = np.asanyarray(Xts)
Xts = Xts.reshape((Xts.shape[0],Xts.shape[1],1))

In [9]:
print(Xts.shape)

(4184, 90, 1)


In [10]:
Ytr = np.asanyarray(Ytr)
Yts = np.asanyarray(Yts)

In [11]:
# Kayan pencere dönüşüm sonrası kalan  veri sayımız
len(Ytr)

6800

In [12]:
# Verilerimizin %70 ' ni eğitime kalanı Teste ayırıyoruz
#k = int(0.8*len(Y))
Xtrain =Xtr
Xtest = Xts

In [13]:
Ytrain = Ytr
Ytest = Yts

In [14]:
Xtrain = Xtrain.astype('float32')
Xtest = Xtest.astype('float32')
Xtrain /=20
Xtest /= 20

In [15]:
print(Ytr)

[[ 1]
 [ 1]
 [ 0]
 ...
 [ 0]
 [ 0]
 [-1]]


In [16]:
# Ağ Yapısını belirliyoruz.

model = Sequential()
'''
model.add(LSTM(64,
               batch_input_shape=(None,timestep,1),     
               return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mse', optimizer='rmsprop',metrics=['accuracy'])
'''
model.add(LSTM(500,input_shape=(timestep,1),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(500, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(500, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation="linear"))
model.compile(loss="mse", optimizer='rmsprop',metrics=['accuracy'])

In [17]:
# Oluşturulan Ağ yapısı hakkında özet bilgiler
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 90, 500)           1004000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 90, 500)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 90, 500)           2002000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 90, 500)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 500)               2002000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 501       
Total para

In [18]:
# Eğitim işlemi kısmı
history=model.fit(Xtrain,
          Ytrain,
          validation_data=(Xtest,Ytest),
          verbose=1,
          epochs=800)

Train on 6800 samples, validate on 4184 samples
Epoch 1/800
6800/6800 [==============================] - 178s 26ms/step - loss: 28.5823 - acc: 0.0918 - val_loss: 25.0385 - val_acc: 0.1035
Epoch 2/800
6800/6800 [==============================] - 172s 25ms/step - loss: 27.4564 - acc: 0.0968 - val_loss: 23.6622 - val_acc: 0.1054
Epoch 3/800
6800/6800 [==============================] - 172s 25ms/step - loss: 27.0803 - acc: 0.0862 - val_loss: 24.5806 - val_acc: 0.0746
Epoch 4/800
6800/6800 [==============================] - 172s 25ms/step - loss: 23.7274 - acc: 0.0968 - val_loss: 19.6085 - val_acc: 0.0727
Epoch 5/800
6800/6800 [==============================] - 172s 25ms/step - loss: 11.5870 - acc: 0.1387 - val_loss: 14.8134 - val_acc: 0.0206
Epoch 6/800
6800/6800 [==============================] - 172s 25ms/step - loss: 6.9483 - acc: 0.1660 - val_loss: 3.9611 - val_acc: 0.2337
Epoch 7/800
6800/6800 [==============================] - 172s 25ms/step - loss: 4.9031 - acc: 0.1919 - val_loss: 6

Epoch 118/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1549 - acc: 0.8521 - val_loss: 3.9758 - val_acc: 0.2158
Epoch 119/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1511 - acc: 0.8459 - val_loss: 3.7298 - val_acc: 0.2378
Epoch 120/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1548 - acc: 0.8472 - val_loss: 3.9463 - val_acc: 0.2211
Epoch 121/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1603 - acc: 0.8401 - val_loss: 3.5446 - val_acc: 0.2321
Epoch 122/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1655 - acc: 0.8479 - val_loss: 3.3772 - val_acc: 0.2175
Epoch 123/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1480 - acc: 0.8532 - val_loss: 3.5285 - val_acc: 0.2409
Epoch 124/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1546 - acc: 0.8616 - val_loss: 3.4976 - val_acc: 0.2409
Epoch 125/800
6800/6

6800/6800 [==============================] - 172s 25ms/step - loss: 0.1026 - acc: 0.9034 - val_loss: 3.7526 - val_acc: 0.2213
Epoch 235/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1031 - acc: 0.9056 - val_loss: 4.1361 - val_acc: 0.1919
Epoch 236/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1116 - acc: 0.8943 - val_loss: 3.8807 - val_acc: 0.2218
Epoch 237/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1077 - acc: 0.9013 - val_loss: 3.8780 - val_acc: 0.2151
Epoch 238/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0984 - acc: 0.9060 - val_loss: 3.8395 - val_acc: 0.2347
Epoch 239/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1029 - acc: 0.9018 - val_loss: 3.7163 - val_acc: 0.2278
Epoch 240/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.1040 - acc: 0.9062 - val_loss: 4.0035 - val_acc: 0.2022
Epoch 241/800
6800/6800 [=========

6800/6800 [==============================] - 172s 25ms/step - loss: 0.0905 - acc: 0.9134 - val_loss: 4.2696 - val_acc: 0.2139
Epoch 351/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0906 - acc: 0.9149 - val_loss: 4.3825 - val_acc: 0.2149
Epoch 352/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0863 - acc: 0.9178 - val_loss: 4.3673 - val_acc: 0.1943
Epoch 353/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0858 - acc: 0.9184 - val_loss: 4.4163 - val_acc: 0.2230
Epoch 354/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0892 - acc: 0.9175 - val_loss: 4.3754 - val_acc: 0.2048
Epoch 355/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0847 - acc: 0.9206 - val_loss: 4.4270 - val_acc: 0.1977
Epoch 356/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0893 - acc: 0.9129 - val_loss: 4.0039 - val_acc: 0.2118
Epoch 357/800
6800/6800 [=========

6800/6800 [==============================] - 172s 25ms/step - loss: 0.0764 - acc: 0.9294 - val_loss: 4.7243 - val_acc: 0.1938
Epoch 467/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0803 - acc: 0.9243 - val_loss: 4.4688 - val_acc: 0.2032
Epoch 468/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0782 - acc: 0.9278 - val_loss: 4.4575 - val_acc: 0.2196
Epoch 469/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0800 - acc: 0.9260 - val_loss: 4.3908 - val_acc: 0.2029
Epoch 470/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0797 - acc: 0.9251 - val_loss: 4.8135 - val_acc: 0.1922
Epoch 471/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0762 - acc: 0.9287 - val_loss: 5.0033 - val_acc: 0.2041
Epoch 472/800
6800/6800 [==============================] - 174s 26ms/step - loss: 0.0787 - acc: 0.9235 - val_loss: 4.6073 - val_acc: 0.2043
Epoch 473/800
6800/6800 [=========

6800/6800 [==============================] - 172s 25ms/step - loss: 0.0769 - acc: 0.9278 - val_loss: 4.5119 - val_acc: 0.2070
Epoch 583/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0722 - acc: 0.9334 - val_loss: 4.3862 - val_acc: 0.2192
Epoch 584/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0746 - acc: 0.9335 - val_loss: 4.4433 - val_acc: 0.2067
Epoch 585/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0752 - acc: 0.9296 - val_loss: 4.5919 - val_acc: 0.1993
Epoch 586/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0737 - acc: 0.9310 - val_loss: 4.4684 - val_acc: 0.2012
Epoch 587/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0742 - acc: 0.9326 - val_loss: 4.3336 - val_acc: 0.2127
Epoch 588/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0721 - acc: 0.9306 - val_loss: 4.4378 - val_acc: 0.2065
Epoch 589/800
6800/6800 [=========

6800/6800 [==============================] - 172s 25ms/step - loss: 0.0696 - acc: 0.9343 - val_loss: 4.3651 - val_acc: 0.2010
Epoch 699/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0708 - acc: 0.9375 - val_loss: 4.5187 - val_acc: 0.1852
Epoch 700/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0689 - acc: 0.9397 - val_loss: 4.3920 - val_acc: 0.1950
Epoch 701/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0706 - acc: 0.9356 - val_loss: 4.3446 - val_acc: 0.1986
Epoch 702/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0708 - acc: 0.9347 - val_loss: 5.2414 - val_acc: 0.1819
Epoch 703/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0713 - acc: 0.9366 - val_loss: 4.3960 - val_acc: 0.2115
Epoch 704/800
6800/6800 [==============================] - 172s 25ms/step - loss: 0.0718 - acc: 0.9325 - val_loss: 4.3033 - val_acc: 0.2067
Epoch 705/800
6800/6800 [=========

In [19]:
score, acc = model.evaluate(Xtest, Ytest)
print('Test score:', score)
print('Test accuracy:', acc)

4184/4184 [==============================] - 21s 5ms/step
Test score: 4.203547808681802
Test accuracy: 0.22586042062160389


In [20]:
model.save('model_ocak_yeni_90timestep.h5')

In [31]:
print(len(Xtest),len(Ytest))
pred=model.predict(Xtest[i].reshape(-1,90,1))
print(pred)

4184 4184
[[2.330851]]


In [67]:
import time
start_time = time.time()
dogru=float(0)
yanlis=0
total=len(Xtest)
hassasiyet=float(2.5)
onceki=float(0)
sonraki=float(0)
for i in range(total):
    pred=model.predict(Xtest[i].reshape(-1,90,1))
    fark=Ytest[i]-pred
    fark=abs(fark)
    fark=int(fark)
    onceki=Ytest[i]-hassasiyet
    sonraki=Ytest[i]+hassasiyet
    #print("tahmin:",pred," Dogru: ",Ytest[i])
    #print("fark: ",fark)
    if(fark<=hassasiyet):
        dogru=dogru+1
        #print("doğru")
    else:
        yanlis=yanlis+1
        #print("yanlış")
    #print("-----------------------------------------------")
        
print('total:',total)
print('dogru:',dogru)
print('yanlis:',yanlis)
yuzde=(dogru*100)/total
print("Doğruluk: ",yuzde)
# your code
elapsed_time = time.time() - start_time
print(elapsed_time)

total: 4184
dogru: 3695.0
yanlis: 489
Doğruluk:  88.31261950286807
777.389405965805


In [46]:
yuzde=(20*100)/40
print("Doğruluk: ",yuzde)

Doğruluk:  50.0


In [ ]:
hassasiyet 1   de   %71
hassasiyet 1.5 de   %71,30
hassasiyet 2   de   %88
hassasiyet 2,5 de   %88.30